In [ ]:
include("_.jl")

In [ ]:
n_ro = 8

n_co = 4

ta_ = collect(1.0:n_co)

da = BioLab.Matrix.simulate(n_ro, n_co, "1.0:")

ro_ = ["Row $id" for id in 1:n_ro]

co_ = ["Column $id" for id in 1:n_co]

st = rand(n_ro, 4)

ht = ""

In [ ]:
using Printf

LAYOUT =
    Dict("width" => 800, "margin" => Dict("l" => 200, "r" => 200), "title" => Dict("x" => 0.5))

ANNOTATION = Dict(
    "yref" => "paper",
    "xref" => "paper",
    "yanchor" => "middle",
    "font" => Dict("size" => 10),
    "showarrow" => false,
)

HEATMAP = Dict("type" => "heatmap", "showscale" => false)

function _make_target_annotation(y, ro)

    return BioLab.Dict.merge(
        ANNOTATION,
        Dict("y" => y, "x" => 0, "xanchor" => "right", "text" => "<b>$ro</b>"),
        BioLab.Dict.set_with_last!,
    )

end

function _get_x(id)

    return 0.97 + id / 6.4

end

function _make_data_annotation(y, la, he, ro_, st)

    annotations = Vector{Dict{String, Any}}()

    if la

        for (id, text) in enumerate(("Sc (⧳)", "Pv", "Ad"))

            push!(
                annotations,
                BioLab.Dict.merge(
                    ANNOTATION,
                    Dict(
                        "y" => y,
                        "x" => _get_x(id),
                        "xanchor" => "center",
                        "text" => "<b>$text</b>",
                    ),
                    BioLab.Dict.set_with_last!,
                ),
            )

        end

    end

    y -= he

    for idy in 1:length(ro_)

        push!(
            annotations,
            BioLab.Dict.merge(
                ANNOTATION,
                Dict(
                    "y" => y,
                    "x" => 0,
                    "xanchor" => "right",
                    "text" => BioLab.String.limit(ro_[idy], 16),
                ),
                BioLab.Dict.set_with_last!,
            ),
        )

        sc, ma, pv, ad = st[idy, :]

        sc = @sprintf("%.3f", sc)

        ma = @sprintf("%.2f", ma)

        pv = @sprintf("%.3f", pv)

        ad = @sprintf("%.3f", ad)

        for (idx, text) in enumerate(("$sc ($ma)", pv, ad))

            push!(
                annotations,
                BioLab.Dict.merge(
                    ANNOTATION,
                    Dict("y" => y, "x" => _get_x(idx), "xanchor" => "center", "text" => text),
                    BioLab.Dict.set_with_last!,
                ),
            )

        end

        y -= he

    end

    return annotations

end

In [ ]:
n_ro, n_co = size(da)

n_ro += 2

he = 1 / n_ro

layout = BioLab.Dict.merge(
    LAYOUT,
    Dict(
        "height" => max(640, 24 * n_ro),
        "title" => Dict("text" => "Match Panel"),
        "yaxis2" =>
            Dict("domain" => (1 - he, 1.0), "showticklabels" => false, "tickvals" => ()),
        "yaxis" =>
            Dict("domain" => (0.0, 1 - he * 2), "showticklabels" => false, "tickvals" => ()),
        "annotations" => vcat(
            _make_target_annotation(1 - he / 2, "Target Name"),
            _make_data_annotation(1 - he / 2 * 3, true, he, ro_, st),
        ),
    ),
    BioLab.Dict.set_with_last!,
)

HEATMAP["x"] = co_

data = [
    BioLab.Dict.merge(
        HEATMAP,
        Dict(
            "yaxis" => "y2",
            "z" => [ta_],
            "text" => co_,
            #"zmin"=> ,
            #"zmax"=> ,
            "colorscale" => BioLab.Plot.fractionate(BioLab.Plot.COPLA),
            "hoverinfo" => "x+z",
        ),
        BioLab.Dict.set_with_last!,
    ),
    BioLab.Dict.merge(
        HEATMAP,
        Dict(
            "yaxis" => "y",
            "y" => ro_,
            "z" => collect(eachrow(da)),
            #"text" => ,
            #"zmin"=> ,
            #"zmax"=> ,
            "colorscale" => BioLab.Plot.fractionate(BioLab.Plot.COPLA),
            "hoverinfo" => "x+y+z",
        ),
        BioLab.Dict.set_with_last!,
    ),
]

BioLab.Plot.plot(data, layout; ht)